In [1]:
from source.regency import Regency
from source.run import rungame
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
X = 5
my_game = rungame('TestGame', train=True, GameLength=X, IntDC = 15)
Game = my_game.Game
Game.change_regent('Is', Player=True)
Game.change_regent('WIT', Player=True)
Game.LeyLines = Game.LeyLines.append(pd.DataFrame([['Is', 'Ywrndor', 'Llewhoellen']], columns=['Regent', 'Province', 'Other']))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Continuing Game TestGame at Season 0 Action 1


In [3]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
Game.take_domain_actions()
Game.take_domain_actions()
Game.take_domain_actions()
Game.LeyLines = Game.LeyLines.append(pd.DataFrame([['Is','Ywrndor', 'Iseare']]))

Someone is trying to perform the following action against Emerald Queen Isaelie [Is]: Espionage Heresy
How much Regency will they spend to counter this?
[RP 106, DC 27]3


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import numpy as np
import pandas as pd
self = Game

Regent, decision, actor, Type, state, capital, high_pop, low_pop, friend, enemy, rando, enemy_capital = Game.errors[2]

In [9]:
Regent = 'Is'
self = Game

temp = self.Holdings[self.Holdings['Regent'] == Regent].copy()
temp = temp[temp['Type']=='Source'][['Type', 'Level', 'Province']]
temp = temp[temp['Level']>=3]
temp = pd.concat([temp[['Province']], self.LeyLines[self.LeyLines['Regent']==Regent][['Province']]])
temp = pd.merge(temp, self.Provinces[['Province', 'Domain', 'Regent']]).drop_duplicates()

In [10]:
temp

,Province,Domain,Regent
0,Annydwr,The Sielwode,Is
1,Cu Haellyrd,The Sielwode,Is
2,Fhylallien,The Sielwode,Is
3,Ghyllwn,The Sielwode,Is
4,Hoehnaen,The Sielwode,Is
5,Iseare,The Sielwode,Is
6,Llewhoellen,The Sielwode,Is
7,Tuar Llyrien,The Sielwode,Is
8,Ywrndor,The Sielwode,Is
